#AS_NZS_4600:2018.py method

This method references the following standard:
AS/NZS 4600:2018, for Cold-formed steel structures.

Method developed 24 May 2022
(c) BVT Consulting Ltd

Developed - SB

Reviewed - MB

**Version summary:**

24.05.2022 - v 0.0 - DRAFT Sections 1.6, 3.1-3.5, 5.3 added

07.07.2022 - v 1.0 - Initial release for use with BVT Selective/Pallet Racking Performance Function. Validated for typical 100x67x2.0 racking post against existing BVT spreadsheets.

## Notes on scope and assumptions

In general, this standard function library currently covers then following sections of AS/NZS 4600:2018. Parts **not** listed below are not covered:

- Table 1.6.3 - Capacity Reduction Factor
- Section 3 - Members
> except:
>> - 3.3.4 - Shear
>> - 3.3.5 - Combined bending and shear 
>> - 3.4.1(6),(6) - G550 steel, less than 0.9 mm BMT
>> - 3.3.6 - Bearing 
>> - 3.3.7 - Combined bending and bearing
>> - 3.3.8 - Stiffeners
>> - 3.6 - Cylindrical members

- Appendix D1.1 - Buckling stresses and moments for members in compression
> except:
>> - D1.1.1.4 - Non-symmetric sections
>> - D1.2 - Distortional buckling stresses
>> - D1.3 - Local buckling stresses

- Appendix D2.1 - Buckling stresses and moments for members in bending
> except:
>> - D2.1.1.2 - Buckling coefficient allowing for moment distribution, $C_b$, conservatively taken as 1.0
>> - D2.2 - Distortional buckling stresses
>> - D2.3 - Local buckling stresses

Calculation of section properties to Section 2, including reduced section properties, is not covered in this performance function and is part of the required input information. 

##Initialise  Dependents and Libraries

In [ ]:
import pandas as pd
import numpy as np
import math

## Input information
NOTE: UNITS ARE N, Nmm, mm

All information is input to functions using one or more of the section, member and connection properties dictionaries.

Dictionaries are input here to use for testing, as well as to set dictionaries - this is required for the functions to run as they are initially imported by other Engineering Methods.

When the functions are called, the relevant dictionaries are provided as part of the input so the information here will not be used.

Information on the input dictionaries can be found here: https://docs.google.com/document/d/1b3DVz-Y_lmYiqm0KnF_Pif5EkcBTiYdxhUq5HGWubO0/edit?usp=sharing 

In [ ]:
# section, member and connection property dicts are set here. When functions are called these are overridden with the information for the item to be checked
# 100x67x2.0 post section properties. In this case X is used for the minor axis
section_properties = {
    'Zx': 6595.5001,
    'Zy': 1.40137e4,
    'Zex': 6247.4937,
    'Zey': 1.32149e4	,
    'Ix': 2.91297e5,
    'Iy': 7.14700e5,
    'J' : 712.071,
    'Iw': 1.14391e9,
    'symmetry axes': 'y',
    'Af': 541.6281,
    'Ae': 494,
    't' : 2,
    'x0': 0,
    'y0': -58.3515,
    'beta': 123.3769,		
    'E' : 200e3,
    'G' : 80e3,
    'fy': 345,
    'fu': 470,
    'hole sections': {
        'holes present?': True,
        'Ix,net': 2.61527e5	,
        'Iy,net': 6.73961e5	,
        'Zx,net': 6247.4937,
        'Zy,net': 1.32149e4,
        'J,net' : 645,
        'Iw,net': 5.2e8,
        'A,net' : 494,
        'x0,net': 0,
        'y0,net': -62.7428,
        'hole length': 25,
        'hole spacing': 76,
        },
    'is subject to distortional buckling?': True,
    'fod' : {'fod_x':574,
               'fod_y':675,
               'fod_c':319
               },
    'is cylinder?': False,
    'bolt hole diameter': 4*14
}

print(section_properties)

# 100x67x2.0 post member - typical values entered
member_properties = {
    'Mx' : 0.151,
    'My' : 0.344,
    'N'  : 30.908,
    'Vx' : 5.301,
    'Vy' : 2.13,
    'lex' : 68.4,
    'ley' : 76,
    'lez' : 76,
    'cantilevered?': False
}

print(member_properties)

# typ M10 bolt to 1.8 mm brace and 2.0 mm posts entered
connection_properties = {
    'connection type' : 'bolted',
    'connection size' : 10,
    'washer diameter' : 16,
    'washer thickness': 2,
    't1' : 1.8,
    'fy1': 350,
    'fu1': 470,
    'e1': 19,
    'w1': 36,
    'An1': (36 + 2*24 - 12)*1.8,
    'alpha1': 0.75,
    't2' : 2,
    'fy2': 350,
    'fu2': 470,
    'e2' : 20,
    'w2' : 67,
    'An2': section_properties['hole sections']['A,net']-12*2,
    'alpha2': 0.75,
    'spacing' : None,
    'V' : 11200,
    'Nt' : 0
}

print(connection_properties)

{'Zx': 6595.5001, 'Zy': 14013.7, 'Zex': 6247.4937, 'Zey': 13214.9, 'Ix': 291297.0, 'Iy': 714700.0, 'J': 712.071, 'Iw': 1143910000.0, 'symmetry axes': 'y', 'Af': 541.6281, 't': 2, 'Ae': 494, 'x0': 0, 'y0': -58.3515, 'beta': 123.3769, 'E': 200000.0, 'G': 80000.0, 'fy': 345, 'fu': 470, 'hole sections': {'holes present?': True, 'Ix,net': 261527.0, 'Iy,net': 673961.0, 'Zx,net': 6247.4937, 'Zy,net': 13214.9, 'J,net': 645, 'Iw,net': 520000000.0, 'A,net': 494, 'x0,net': 0, 'y0,net': -62.7428, 'hole length': 25, 'hole spacing': 76}, 'is subject to distortional buckling?': True, 'fod': {'fod_x': 574, 'fod_y': 675, 'fod_c': 319}, 'is cylinder?': False, 'bolt hole diameter': 56}
{'Mx': 0.151, 'My': 0.344, 'N': 30.908, 'Vx': 5.301, 'Vy': 2.13, 'lex': 68.4, 'ley': 76, 'lez': 76, 'cantilevered?': False}
{'connection type': 'bolted', 'connection size': 10, 'washer diameter': 16, 'washer thickness': 2, 't1': 1.8, 'fy1': 350, 'fu1': 470, 'e1': 19, 'w1': 36, 'An1': 129.6, 'alpha1': 0.75, 't2': 2, 'fy2': 

# Axis setter

Information is input based on section x and y axis. This function converts values to major or minor axis, allowing sections to be input in any rotation.

Major axis is taken as the one with the highest stiffness (highest $I$ value)

In [ ]:
def axis_setter(section_properties):
  if section_properties['Ix'] >= section_properties['Iy']:
    section_properties['major_axis'] = 'x'
    section_properties['Imaj'] = section_properties['Ix']
    section_properties['Zf_maj'] = section_properties['Zx']
    section_properties['Ze_maj'] = section_properties['Zex']
    section_properties['shear_centre_maj'] = section_properties['x0'] 
    section_properties['Imin'] = section_properties['Iy']
    section_properties['Zf_min'] = section_properties['Zy']
    section_properties['Ze_min'] = section_properties['Zey']
    section_properties['shear_centre_min'] = section_properties['y0'] 
    section_properties['hole sections']['Imaj,net'] = section_properties['hole sections']['Ix,net']
    section_properties['hole sections']['Imin,net'] = section_properties['hole sections']['Iy,net']
    section_properties['hole sections']['Zmaj,net'] = section_properties['hole sections']['Zx,net']
    section_properties['hole sections']['Zmin,net'] = section_properties['hole sections']['Zy,net']
    section_properties['hole sections']['shear_centre_maj,net'] = section_properties['hole sections']['x0,net']
    section_properties['hole sections']['shear_centre_min,net'] = section_properties['hole sections']['y0,net']
    section_properties['fomaj'] = section_properties['fod']['fod_x']
    section_properties['fomin'] = section_properties['fod']['fod_y']
    section_properties['foc'] = section_properties['fod']['fod_c']

    if section_properties['symmetry axes'] =='x':
      section_properties['symmetry axes maj min'] = 'maj'
    elif section_properties['symmetry axes'] =='y':
      section_properties['symmetry axes maj min'] = 'min'
    elif section_properties['symmetry axes'] =='double':
      section_properties['symmetry axes maj min'] = 'double'

  else:
    section_properties['major_axis'] = 'y'
    section_properties['Imaj'] = section_properties['Iy']
    section_properties['Zf_maj'] = section_properties['Zy']
    section_properties['Ze_maj'] = section_properties['Zey']
    section_properties['shear_centre_maj'] = section_properties['y0'] 
    section_properties['Imin'] = section_properties['Ix']
    section_properties['Zf_min'] = section_properties['Zx']
    section_properties['Ze_min'] = section_properties['Zex']
    section_properties['shear_centre_min'] = section_properties['x0'] 
    section_properties['hole sections']['Imaj,net'] = section_properties['hole sections']['Iy,net']
    section_properties['hole sections']['Imin,net'] = section_properties['hole sections']['Ix,net']
    section_properties['hole sections']['Zmaj,net'] = section_properties['hole sections']['Zy,net']
    section_properties['hole sections']['Zmin,net'] = section_properties['hole sections']['Zx,net']
    section_properties['hole sections']['shear_centre_maj,net'] = section_properties['hole sections']['y0,net']
    section_properties['hole sections']['shear_centre_min,net'] = section_properties['hole sections']['x0,net']
    section_properties['fomaj'] = section_properties['fod']['fod_y']
    section_properties['fomin'] = section_properties['fod']['fod_x']
    section_properties['foc'] = section_properties['fod']['fod_c']

    if section_properties['symmetry axes'] =='y':
      section_properties['symmetry axes maj min'] = 'maj'
    elif section_properties['symmetry axes'] =='x':
      section_properties['symmetry axes maj min'] = 'min'
    elif section_properties['symmetry axes'] =='double':
      section_properties['symmetry axes maj min'] = 'double'

  return section_properties

# Section Reviewer
Reviews section_properties dictionary and returns an error if the section is outside of the scope of this performance function.


In [ ]:
def section_reviewer(section_properties):
  errors = []

  # check if non-symmetric section. Major axis bending not covered as AS/NZS 4600:2018 does not contain a method for assessing lateral buckling of these sections.
  if section_properties['symmetry axes'] == 'none':
    errors.append('Buckling checks for members with no symmetry axes not covered in this function library. Other analysis required')
  elif section_properties['symmetry axes'] == 'Z point symmetric':
    errors.append('Calculation of elastic lateral-torsional buckling moment for point symmetric Z sections not yet covered in performance function. Manual calculation using Appendix D2.1.1.3 required')
  elif section_properties['symmetry axes'] in {'x','y','double'}:
     'filler - no action'
  else:
    errors.append('"symmetry axes" input not recognised. Input should be string of either "x", "y", "double", "Z point symmetric" or "none"')

  #check for G550 steel, under 0.9 mm BMT
  if section_properties['fy'] >= 550 and section_properties['t'] < 0.9:
    errors.append('Error: additional requirements for G550 steel, less than 0.9 mm BMT given in Section 3.4.1 are not covered in this function library')

  #check for G550 steel, under 0.9 mm BMT
  if section_properties['is cylinder?'] == True:
    errors.append('Error: Member capacity calculations for cylinders as given in Section 3.6 are not covered in this function library')

  return errors

errors = section_reviewer(section_properties)
errors

[]

#1. Scope and General

## 1.6.3 Capacity reduction factor

Only the values used in the sections covered by this performance funtion are included. Further values can be added as required as the performance function is developed to cover more of AS/NZS 4600:2018.

Note that in some cases where a range of capacity reduction factors are given, the lower value is used. These are:

- Combined axial and beanding: Bending
- Bolted connections: Tension

In [ ]:
table1_6_3 = pd.DataFrame(
   [['Members subject to axial tension','Members subject to axial tension', "3.2", 0.90],
    ['Members subject to bending',"Section moment capacity - for sections with stiffened or partially stiffened compression flanges", "3.3.2", 0.95],
    ['Members subject to bending',"Section moment capacity - for sections with unstiffened compression flanges", "3.3.2", 0.90],
    ['Members subject to bending',"Member moment capacity - lateral and distortional buckling and beams with one flange through fastened to sheeting", "3.3.3", 0.90],
    ['Members subject to bending',"Web design - shear", "3.3.4", 0.90],
    ['Concentrically loaded compression members',"All", "3.4", 0.85],
    ['Combined axial load and bending',"Compression", "3.5.1", 0.85],
    ['Combined axial load and bending',"Bending", "3.5.1", 0.90],
    ['Bolted connections',"Tearout", "5.3.2", 0.6],
    ['Bolted connections',"Net section tension", "5.3.3", 0.8],
    ['Bolted connections',"Bearing", "5.3.4", 0.6],
    ['Bolted connections',"Bolt - in shear", "5.3.5.1", 0.8],
    ['Bolted connections',"Bolt - in tension", "5.3.5.2", 0.8],
   ], 
columns = ["category", "Design capacity of:","Reference","Capacity Reduction Factor"])

table1_6_3

,category,Design capacity of:,Reference,Capacity Reduction Factor
0,Members subject to axial tension,Members subject to axial tension,3.2,0.90
1,Members subject to bending,Section moment capacity - for sections with st...,3.3.2,0.95
2,Members subject to bending,Section moment capacity - for sections with un...,3.3.2,0.90
3,Members subject to bending,Member moment capacity - lateral and distortio...,3.3.3,0.90
4,Members subject to bending,Web design - shear,3.3.4,0.90
5,Concentrically loaded compression members,All,3.4,0.85
6,Combined axial load and bending,Compression,3.5.1,0.85
7,Combined axial load and bending,Bending,3.5.1,0.90
8,Bolted connections,Tearout,5.3.2,0.60
9,Bolted connections,Net section tension,5.3.3,0.80


# 3. Members

##3.2 Members subject to axial tension

### 3.2.1 Design for axial tension

>> $ N^* \le \phi_tN_t
$

The tension unity for the member is calculated:

>> $ TensionUnity = \frac{N^*}{\phi_tN_t}
$

In [ ]:
def Clause_3_2_1_tension_unity(section_properties, member_properties, **k_t):
  # get phi_t from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 3.2, .iloc[0] takes the first value of this series, which is the phi factor
  phi_t = table1_6_3.loc[table1_6_3['Reference'] == '3.2','Capacity Reduction Factor'].iloc[0]

  # calculate N_t using the function for nominal section tension capacity
  N_t = nominal_section_tension_capacity(section_properties)

  # get tension action. 0 tension action will be returned if axial action is negative (i.e. compression)
  if member_properties['N'] >= 0:
    N_action = member_properties['N']
  else:
    N_action = 0

  #calculate tension unity
  tension_unity = N_action/(phi_t*N_t)
  
  return tension_unity

### 3.2.2 Nominal section capacity

Takes the lesser of equations 3.2.2(1) and 3.2.2(2)

In [ ]:
def nominal_section_tension_capacity(section_properties,kt=0.75):
  kt = tension_connection_factor(kt)

  # get section, material properties
  Ag = section_properties['Af']
  fy = section_properties['fy']
  fu = section_properties['fu']
  
  # get net area by taking gross area (non-holed sections) or holed net area (holed sections) and subtracting the fastener hole diameter if applicable
  if section_properties['hole sections']['holes present?']== True:
    An1 = section_properties['hole sections']['A,net']
  else:
    An1 = Ag

  A_bolt_holes = section_properties['bolt hole diameter'] * section_properties['t'] 
  An = An1 - A_bolt_holes

  N_t = min(
      Ag*fy,
      0.85*kt*An*fu
  )
  return N_t

### 3.2.3 Distribution of forces

The k_t factor is taken as 0.75 by default as this is the minimum value given by Table 3.2 (see below). Higher values may be input where applicable.

In [ ]:
def tension_connection_factor(kt = 0.75):
  kt = kt
  return kt

## 3.3 Members subject to bending

### 3.3.1 Bending moment

Design bending moment shall be less than the factored limits for both section moment and member moment:

>> $ M^*\le \phi_bM_s
$

>> $M^*\le \phi_bM_b
$



In [ ]:
def factored_section_bending_capacity(section_properties, member_properties, axis):  
  # for section bending there are two options in Table 1.6.3 for phi. The minimum is taken as members being checked (steel stud, racking etc) often do not have stiffened compression flanges.
  phi_b_s = table1_6_3.loc[table1_6_3['Reference'] == '3.3.2','Capacity Reduction Factor']
  phi_b_s = min(phi_b_s.iloc[0],phi_b_s.iloc[1])
  
  # calculate M_s using the function for nominal section bending capacity
  Msx,Msy = nominal_section_moment_capacity(section_properties)
  if axis == 'x':
    Ms = Msx
  if axis == 'y':
    Ms = Msy

  return phi_b_s*Ms

In [ ]:
def factored_member_bending_capacity(section_properties, member_properties, axis):    
  # get phi_b for section and member bending from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 3.3.3, .iloc[0] takes the first value of this series, which is the phi factor
  phi_b_m = table1_6_3.loc[table1_6_3['Reference'] == '3.3.3','Capacity Reduction Factor'].iloc[0]
  
  # calculate M_s using the function for nominal section bending capacity
  Msx,Msy = nominal_section_moment_capacity(section_properties)
  if axis == 'x':
    Ms = Msx
  if axis == 'y':
    Ms = Msy
 
  # get M_b using the function for nominal section bending capacity. 
  Mb = nominal_member_bending_capacity(section_properties, member_properties, axis, Ms)

  return phi_b_m*Mb

The bending unity is calculated:

>> $ BendingUnity = \frac{M^*}{min(\phi_bM_s,\phi_bM_b)}
$

Note that the $\phi$ factor for section moment capacity is currently taken as the minimum out of the options given in Table 1.6.3. If the compression flange is stiffened a higher factor may be used. 

In [ ]:
def Clause_3_3_bending_unity(section_properties, member_properties, axis):
  
  # get bending action for correct axis. Moments converted to absolute values
  if axis == 'x':
    M_action = abs(member_properties['Mx'])
  if axis == 'y':
    M_action = abs(member_properties['My'])

  # get factored section and member capacities
  phi_Ms = factored_section_bending_capacity(section_properties, member_properties, axis)
  phi_Mb = factored_member_bending_capacity(section_properties, member_properties, axis)

  # calculate bending unity. 
  bending_unity = M_action/min(phi_Mb,phi_Ms)
  
  return bending_unity

#### 3.3.2 Nominal section moment capacity

The nominal section capacity based on initiation of yeilding is taken per 3.3.2.2:

>> $M_s = Z_ef_y
$

Slighly higher capacity is possible for qualifying sections under 3.3.2.3 by utilising inelastic reserve capacity. However in most cases members under consideration are subject to buckling which disqualify them from this method.



In [ ]:
def nominal_section_moment_capacity(section_properties):
  Zex = section_properties["Zex"]
  Zey = section_properties["Zey"]
  fy = section_properties["fy"]
  M_sx = Zex*fy
  M_sy = Zey*fy
  return M_sx, M_sy

### 3.3.3 Nominal member moment capacity

Nominal member moment capacity is taken as the minimum of nominal section moment capacity and values calculated using sections 3.3.3.2 and 3.3.3.3.

Note that section 3.3.3.4 is not considered - this can give extra strength where one flange is fixed to a sheet. This applies to purlins fixed to roofing sheet, as well as partitions lined one side.

In [ ]:
def nominal_member_bending_capacity(section_properties, member_properties, axis, Ms):
  # set major/ minor axes
  section_properties = axis_setter(section_properties)
  
  # find limiting capacity out of lateral and distortional buckling. If section is set as not subject to distortional buckling, this value is set as equal to the lateral buckling capacity
  if axis != section_properties['major_axis']:
    # if we are considering minor axis bending take Mb_lat as infinity as it is not a failure mode
    Mb_lat = Ms
  elif axis == section_properties['major_axis'] and section_properties['symmetry axes'] in {'x','y','double'}:
    # if section is symmetrical and under major axis bending then lateral buckling is assessed per Section 3.3.3.2
    Mb_lat = nominal_member_moment_capacity_lateral(section_properties, member_properties)
  
  if section_properties['is subject to distortional buckling?'] == True:
    Mb_dist = nominal_member_moment_capacity_distortional(section_properties,axis)
  elif section_properties['is subject to distortional buckling?'] == False:
    # if section is not subject to distortional buckling take Mb_dist as infinity as it is not a failure mode
    Mb_dist = Ms

  Mb = min(Mb_dist,Mb_lat, Ms)

  return Mb

####3.3.3.2 Members subject to lateral buckling

Only open, symmetric sections are considered here, using section 3.3.3.2.1. Closed sections should be assessed using section 3.3.3.2.2 - this is not yet included in the performance function. Non-symmetric sections are not covered in Section 3.3.3.2.

Equation 3.3.3.2.1(1) calculates member moment capacity based on the section modulus, $Z_c$ for the stress at the extreme compression fibre. For simplicity (and conservativeness), we use the section modulus at yield, $Z_e$, here:

 >> $M_b=Z_cf_c
  $

  Note that this section is considered only for bending in the major axis. If a member is under minor axis bending with the load well above the shear centre then lateral buckling should be considered (https://drive.google.com/file/d/1VKn5xiGkKXTx5yUIZjhEseuQ2k_iPJ8x/view?usp=sharing)

In [ ]:
def nominal_member_moment_capacity_lateral(section_properties, member_properties):
  # this part only applies to symmetrical sections. if statement used to filter out incorrect use for non-symmetrical sections
  if section_properties['symmetry axes'] == 'none':
    Mb = 'Appendix D does not apply - M_o to be calculated by rational analysis'
  elif section_properties['symmetry axes'] in {'x','y','double'}:
    #get major axis Ze
    section_properties = axis_setter(section_properties)
    Ze_maj = section_properties['Ze_maj']

    # get critical uckling stress
    fc = critical_bending_stress_lateral(section_properties, member_properties)

    # get nominal member buckling capacity
    Mb = Ze_maj*fc
  
  else:
    Mb = 'Symmetry axis incorrectly specified. Allowed values are: "x","y","double", "none"'

  return Mb

The stress $f_c$ is calculated using equation 3.3.3.2.1(2):

$f_c = \frac{M_c}{Z_f}
$

The critical moment, $M_c$, is calculated using equestions 3.3.3.2.1(3,4,5):

>  For $\lambda_b\le 0.60: \qquad\qquad\quad M_c=M_y
$

> For $ 0.60<\lambda_b< 1.336: \>\> \quad M_c=1.11M_y[1-\frac{10\lambda_b^2}{36}]
$

>  For $\lambda_b\ge 1.336: \qquad\qquad\>\>\> M_c=M_y(\frac{1}{\lambda_b^2})
$

where:

> $\lambda_b = \sqrt{\frac{M_y}{M_o}} $

>> $M_y = Z_ff_y $

>> $M_o =$  as per Paragraph D2.1, Appendix D 

In [ ]:
def critical_bending_stress_lateral(section_properties, member_properties):
  
  section_properties = axis_setter(section_properties)

  if section_properties['hole sections']['holes present?'] == False:
    Zf = section_properties['Zf_maj']
  else:
    Zf = section_properties['hole sections']['Zmaj,net']
  
  Mc = critical_bending_moment_lateral(section_properties, member_properties)

  fc = Mc/Zf
  return fc

In [ ]:
def critical_bending_moment_lateral(section_properties, member_properties):

  # get major axis full section modulus
  section_properties = axis_setter(section_properties)
  if section_properties['hole sections']['holes present?'] == False:
    Zf = section_properties['Zf_maj']
  else:
    Zf = section_properties['hole sections']['Zmaj,net']

  # get yield strength
  fy = section_properties['fy']

  # get elastic buckling moment
  # this part only applies to symmetrical sections. if statement used to filter out incorrect use for non-symmetrical sections
  if section_properties['symmetry axes'] == 'none':
    M_o = 'Appendix D does not apply - M_o to be calculated by rational analysis'
  elif section_properties['symmetry axes'] in {'x','y','double'}:
    M_o = elastic_lateral_torsional_buckling_moment(section_properties, member_properties)
  else:
    M_o = 'Symmetry axis incorrectly specified'

  # get yield moment using eq. 3.3.3.2.1(7)
  axis = section_properties['major_axis']
  M_yield = yield_bending_moment(section_properties,axis)

  # calculate slenderness ratio using eq 3.3.3.2.1(6)
  lambda_b = (M_yield/M_o)**0.5

  if lambda_b <= 0.6:
    M_c = M_yield
  elif lambda_b < 1.336:
    M_c = 1.11*M_yield*(1-(10*lambda_b**2/36))
  else:
    M_c = M_yield*(1/lambda_b**2)

  return M_c

In [ ]:
def yield_bending_moment(section_properties,axis):
  # get section properties for correct axis
  fy = section_properties['fy']
  if axis == 'x':
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zx,net']
    else:
      Zf = section_properties['Zx']
  elif axis == 'y':
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zy,net']
    else:
      Zf = section_properties['Zy']
  
  
  # calvulate yield moment using eq. 3.3.3.2.1(7)
  M_yield = Zf*fy

  return M_yield

####3.3.3.3 Members subject to distortional buckling

Nominal member bending capacity, $M_b$, of sections subject to distortional buckling is calculated using eq. 3.3.3.3(1):

$\quad M_b=Z_cf_c
$

For the simplicity and conservativeness, $Z_e$ is used in place of $Z_c$

Only Section 3.3.3.3(a) is covered here. This applies to most of the sections likely to be used, where the distortional buckling mode is bending of a flange. Section 3.3.3.3(b) covers the case where the flanges remain in plane but move laterally, and the web bends. This generally only applies where the flange has high torsional stiffness, for example where the flanges are a hollow section.

For Section 3.3.3.3(a):

$\quad f_c = \frac{M_c}{Z_f}
$

> where:

> $\quad M_c$ = critical moment

>$ \quad Z_f$ = full section modulus

The critical moment, $M_c$ is calaculated using eqs 3.3.3.3(3) and (4):

> For $\quad \lambda_d\le 0.674:\quad M_c=M_y
$

> For $\quad \lambda_d> 0.674:\quad M_c=\frac{M_y}{\lambda_d}(1-\frac{0.22}{\lambda_d})
$

>> where:

>>$\quad \lambda_d=\sqrt{\frac{M_y}{M_{od}}} \qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad (eq 3.3.3.3(8))
$

>>$\quad M_y=$ moment causing inital yield at the extreme compression fibre of the full section.

>>>$\quad\ = Z_ff_y\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad (eq 3.3.3.3.2.1(7))
$

>>$\quad M_od=$ elastic buckling moment in the distortional mode.

>>>$\quad\ = Z_ff_{od}
$

>>$\quad f_od=$ elastic distortional buckling stress determined in accordance witha rational elastic buckling analysis or Paragraph D2.2, Appendix D

Calculation of the elastic distortional buckling stress, $f_{od}$, per Appendix D2.2, is not currently covered in the AS/NZS 4600:2018 performance function and is instead entered as a property of a section. 

When new sections are created for use with the performance function, the elastic distortional buckling stress, $f_{od}$, should be calulated using AS/NZS 4600:2018 Appendix D2.2 or using FEA software such as ANSYS or CUFSM. Note that CUFSM does not allow for holes in sections - however equation D2.2.2(2) may be used to calaculate a modified thickness, $t_r$:

$ \quad t_r = (\frac{A_{web,net}}{A_{web,gross}})^{\frac{1}{3}}*t
$

Distortional buckling performance is independant of restraint spacing, so long as restraint spacing is greater than the distortional buckling half wavelength (denoted by $\lambda$ in Appendix D1.2)

In [ ]:
def nominal_member_moment_capacity_distortional(section_properties,axis):
  # get section properties for correct axis
  if axis == 'x':
    Zc = section_properties['Zex']
  elif axis == 'y':
    Zc = section_properties['Zey']

  # get critical stress
  fc = critical_bending_stress_distortional(section_properties, axis)

  # calculate Mb
  Mb = Zc*fc

  return Mb

In [ ]:
def critical_bending_stress_distortional(section_properties, axis):
  # get section properties for correct axis
  if axis == 'x':
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zx,net']
    else:
      Zf = section_properties['Zx']
  elif axis == 'y':
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zy,net']
    else:
      Zf = section_properties['Zy']

  # get critical bending moment
  Mc = critical_bending_moment_distortional(section_properties, axis)

  # calculate critical bending stress
  fc = Mc/Zf

  return fc

In [ ]:
def critical_bending_moment_distortional(section_properties, axis):
  # get slenderness ratio
  lambda_d = slenderness_distortional(section_properties, axis)

  # get yield bending moment
  Myield = yield_bending_moment(section_properties,axis)

  # calculate critical moment using eqs 3.3.3.3(3),(4)
  if lambda_d <= 0.674:
    Mc = Myield
  elif lambda_d > 0.674:
    Mc = (Myield/lambda_d)*(1-(0.22/lambda_d))

  return Mc

In [ ]:
def slenderness_distortional(section_properties, axis):
  # get yield moment
  Myield = yield_bending_moment(section_properties,axis)

  # get elastic buckling moment in distortional buckling
  Mod = buckling_moment_distortional(section_properties, axis)

  # calculate slenderness ratio using eq 3.3.3.3(8)
  lambda_d = (Myield/Mod)**0.5

  return lambda_d

In [ ]:
def buckling_moment_distortional(section_properties, axis):
  # get section properties for correct axis
  if axis == 'x':
    fod = section_properties['fod']['fod_x']
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zx,net']
    else:
      Zf = section_properties['Zx']
  elif axis == 'y':
    fod = section_properties['fod']['fod_y']
    # full section modulus taken as Z,net where holes are present    
    if section_properties['hole sections']['holes present?'] == True:
      Zf = section_properties['hole sections']['Zy,net']
    else:
      Zf = section_properties['Zy']

  # calculate elastic buckling moment in distortional mode using eq 3.3.3.3(9)
  Mod = Zf*fod

  return Mod

## 3.4 Concentrically loaded compression members

### 3.4.1 General

The design axial compressive force should satisfy the following:

(a)$\quad N*\le\phi_cN_s
$

(b)$\quad N*\le\phi_cN_c
$

where:

> $\phi_c=$ capacity reduction factor per table 1.6.3

>$N_s=$ nominal section capacityof the member in compression

>$\quad\ = A_ef_y$

>$\qquad\ \ A_e =$ effective area at yield stress ($f_y$)

>$N_c =$ nominal member capacity of the member in compression

>$\quad\ = A_ef_n$

>$\qquad\ \ A_e =$ effective area at critical stress ($f_n$). For simplicity and conservativeness this is taken as effictive area at yield stress here

>$\qquad\ \ f_n =$ critical stress:

>>> For $\lambda_c \ le 1.5: f_n=(0.658^{\lambda_c^2})f_y
$

>>> For $\lambda_c > 1.5: f_n=(\frac{0.877}{\lambda_c^2})f_y
$

>> where:

>>> $\lambda_c= \sqrt{\frac{f_y}{f_{oc}}}$

>>> $f_{oc}=$ least of the elastic flexural, torsional and flexural-torsional buckling stresses. For this performance function this is provided as part of the section properties, and must be calculated when defining the section.



In [ ]:
def Clause_3_4_compression_unity(section_properties,member_properties):
  # get phi_c from Table 1.6.3
  phi_c = table1_6_3.loc[table1_6_3['Reference'] == '3.4','Capacity Reduction Factor']
  phi_c = phi_c.iloc[0]

  # get section compression capacity
  Ns = nominal_section_compression_capacity(section_properties)

  # get member compression capacity
  Nc = nominal_member_compression_capacity(section_properties, member_properties)

  # get compression action. If N action is positive this indicates member tension, so compression is zero. Negative action is converted to positive
  if member_properties['N'] <= 0:
    N_action = -member_properties['N']
  else:
    N_action = 0
    
  # unity per 3.4.1(a)
  unity_a = N_action/(phi_c*Ns)

  # unity per 3.4.1(b)
  unity_b = N_action/(phi_c*Nc)

  # governing unity
  compression_unity = max(unity_a, unity_b)

  return compression_unity

In [ ]:
def nominal_section_compression_capacity(section_properties):
  # get Ae, fy from section properties
  Ae = section_properties['Ae']
  fy =section_properties['fy']

  # calculate Ns using eq 3.4.1(1)
  Ns = Ae*fy

  return Ns

In [ ]:
def nominal_member_compression_capacity(section_properties, member_properties):
  # get Ae, fy, foc from section properties
  Ae = section_properties['Ae']
  fy =section_properties['fy']

  # get fn
  fn = critical_stress_compression(section_properties, member_properties)

  # calculate Nc using eq 3.4.1(2)
  Nc = Ae*fn

  return Nc

In [ ]:
def critical_stress_compression(section_properties, member_properties):
  # get fy from section properties
  fy =section_properties['fy']

  # get lambda_c 
  lambda_c = slenderness_compression(section_properties, member_properties)

  if lambda_c <= 1.5:
    # eq 3.4.1(3)
    fn =(0.658**((lambda_c)**2))*fy
  else:
    # eq 3.4.1(4)
    fn = (0.877/(lambda_c**2))*fy

  return fn

In [ ]:
def slenderness_compression(section_properties, member_properties):
  # get fy from section properties
  fy =section_properties['fy']

  # get elastic flexural buckling stress 
  foc = elastic_flexural_buckling_stress(section_properties, member_properties)

  # calculate lambda_c using eq 3.4.1(5)
  lambda_c = (fy/foc)**0.5
  
  return lambda_c

## 3.5 Combined Axial compression or tension, and bending

Function below can be called when it is unknown whether the member will be in tension or compression. It will selct the correct unity equations to evaluate and return that unity value.

In [ ]:
def Clause_3_5_combined_axial_compression_or_tension_and_bending(section_properties,member_properties):
  if member_properties['N'] <= 0:
    # if axial load is negative (i.e. compression) use Section 3.5.1. This also includes cases with no axial load although this is not defined in the standard
    unity = combined_bending_compression(section_properties,member_properties)
  else:
    # if axial load is positive (i.e. tension) use Section 3.5.2.
    unity = combined_bending_tension(section_properties,member_properties)
  
  return unity

### 3.5.1 Combined axial compression and bending

Combined axial compression and bending is checked using eqs 3.5.1(1),(2),(3).

$C_{mx}$, $C_{my}$ are taken as 1.0 for cantilevered members (as one end will be unrestrained), and 0.85 for all other members for simplicity and conservativeness. 

$\alpha_{nx}$, $\alpha_{ny}$ are calculated in functions defined here.

$N_c$,$N_s$, $M_{bx}$, $M_{by}$, $N_c$,$N_s$ are calaculated using the functions defined in Sections 3.3 and 3.4

In [ ]:
def combined_bending_compression(section_properties,member_properties):
  # get phi_b for member bending from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 3.2, .iloc[0] takes the first value of this series, which is the phi factor
  phi_b_m = table1_6_3.loc[table1_6_3['Reference'] == '3.3.3','Capacity Reduction Factor'].iloc[0]

  # get phi_c from Table 1.6.3
  phi_c = table1_6_3.loc[table1_6_3['Reference'] == '3.4','Capacity Reduction Factor']
  phi_c = phi_c.iloc[0]

  # set Cmx and Cmy. This is 0.85 unless 'is cantilever?' input is True, in which case one end is unrestrained therefore C_m = 1
  if member_properties['cantilevered?'] == True:
    Cmx, Cmy = 1,1
  else:
    Cmx, Cmy = 0.85, 0.85

  # get moment amplification factors 
  alpha_nx = moment_amplification_factor(section_properties,member_properties, 'x')
  alpha_ny = moment_amplification_factor(section_properties,member_properties, 'y')

  # get actions. No compression action will be returned if axial action is postive (i.e. tension). Moments are converted to absolute values at this point
  if member_properties['N'] <= 0:
    N_action = -member_properties['N']
  
  Mx_action = abs(member_properties['Mx'])
  My_action = abs(member_properties['My'])

  # get capacities from Sections 3.3, 3.4. 
  Nc = nominal_member_compression_capacity(section_properties, member_properties)
  Ns = nominal_section_compression_capacity(section_properties)
  Mbx = nominal_member_bending_capacity(section_properties, member_properties, 'x', nominal_section_moment_capacity(section_properties)[0])
  Mby = nominal_member_bending_capacity(section_properties, member_properties, 'y', nominal_section_moment_capacity(section_properties)[1])

  # calculate unities
  if N_action/(phi_c*Nc) > 0.15:
    # eq 3.5.1(1)
    unity_1 = (N_action/(phi_c*Nc)) + (Cmx*Mx_action/(phi_b_m*Mbx*alpha_nx)) + (Cmy*My_action/(phi_b_m*Mby*alpha_ny))

    # eq 3.5.1(2)
    unity_2 = (N_action/(phi_c*Ns)) + (Mx_action/(phi_b_m*Mbx)) + (My_action/(phi_b_m*Mby))

    unity = min(unity_1, unity_2)

  else:
    # eq 3.5.1(3)
    unity = (N_action/(phi_c*Nc)) + (Mx_action/(phi_b_m*Mbx)) + (My_action/(phi_b_m*Mby))

  return unity

In [ ]:
def moment_amplification_factor(section_properties,member_properties, axis):
  # get axis dependent properties
  if axis == 'x':
    Ib = section_properties['Ix']
    leb = member_properties['lex']
  elif axis == 'y':
    Ib = section_properties['Iy']
    leb = member_properties['ley']

  # get non axis dependant properties
  E = section_properties['E']
  
  # get compression action, and turn to positive number. If axial force in member is poitive (i.e. tension) no result will be returned, as this function is for compression only
  if member_properties['N'] <= 0:
    N_action = -member_properties['N']

  # calculate elastic buckling load using eq 3.5.1(6)
  Ne = (E*Ib*math.pi**2)/(leb**2)

  # calculate moment amplification factor using eq 3.5.1(5)
  alphan = 1-(N_action/Ne)

  return alphan


### 3.5.2 Combined axial tension and bending

Combined axial tension and bending is checked using eqs 3.5.2(1),(2).

$M_{sxf}$, $M_{syf}$ are calulated per eq 3.5.2(3)

$N_t$, $M_{bx}$, $M_{by}$, $N_c$,$N_s$ are calaculated using the functions defined in Sections 3.3 and 3.4

In [ ]:
def combined_bending_tension(section_properties,member_properties):
  # get phi_b for member bending from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 3.2, .iloc[0] takes the first value of this series, which is the phi factor
  phi_b_m = table1_6_3.loc[table1_6_3['Reference'] == '3.3.3','Capacity Reduction Factor'].iloc[0]

  # get phi_t from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 3.2, .iloc[0] takes the first value of this series, which is the phi factor
  phi_t = table1_6_3.loc[table1_6_3['Reference'] == '3.2','Capacity Reduction Factor'].iloc[0]

  # get actions. No tension action will be returned if axial action is negative (i.e. compression). Moments converted to absolute values here
  if member_properties['N'] >= 0:
    N_action = member_properties['N']
  
  Mx_action = abs(member_properties['Mx'])
  My_action = abs(member_properties['My'])

  # calculate full section yield moments with eq 3.5.2(3)
  # get full section moduli
  Zxft, Zyft = section_properties['Zx'], section_properties['Zy']
  fy = section_properties['fy']
  Msxf = Zxft*fy
  Msyf = Zyft*fy

  # get capacities from Sections 3.3, 3.4. 
  Nt = nominal_section_tension_capacity(section_properties,kt=0.75)
  Mbx = nominal_member_bending_capacity(section_properties, member_properties, 'x', nominal_section_moment_capacity(section_properties)[0])
  Mby = nominal_member_bending_capacity(section_properties, member_properties, 'y', nominal_section_moment_capacity(section_properties)[1])

  # calculate unities
  unity1 = (Mx_action/(phi_b_m*Mbx))  + (My_action/(phi_b_m*Mby))  - (N_action/(phi_t*Nt))
  unity2 = (Mx_action/(phi_b_m*Msxf)) + (My_action/(phi_b_m*Msyf)) + (N_action/(phi_t*Nt)) 

  unity = max(unity1, unity2)

  return unity

# 5. Connections

A main function is defined for both shear and tension capacity to ensure the correct connection type is checked.

In [ ]:
def Clauses_5_2_and_5_3_connection_shear_unity(connection_properties):
  # get connection type
  connection_type = connection_properties['connection type']
  
  if connection_type == 'welded':
    unity = 'Error - welded connections not currently covered'
  elif connection_type == 'bolted':
    unity = bolted_connection_shear_unity(connection_properties)
  elif connection_type == 'screwed':
    unity = 'Error - Screwed connections not currently covered'
  else:
    unity = 'Error - Connection type not recognised'

  return unity

##5.2 Welded connections

Not covered yet

## 5.3 Bolted Connections

The geometry requirements given in Section 5.3.1 are not checked here.

Strength of the bolt itself is not evaluated, as this does not typically limit for connections in thin cold formed steel.

The shear capacity for bolted connections is calculated based on the worst case of:

- Tearout per Section 5.3.2
- Net section tension per Section 5.3.3
- Bearing per Section 5.3.4 (limited hole deformation per Section 5.3.4.3 is not currently covered)

The tension capacity is also calculated based on the pull-over/ pull-through capacity for screws per Section 5.4.3.2. Although this is given for screws it is conservative to use for bolts so long as washers are used.

Note that this covers capacity for a single bolt. In cases where connections have multiple bolts, the load per bolt will be calculated by the relevant performance function and compared against the capacity calculated here.

In [ ]:
def bolted_connection_shear_unity(connection_properties):
  unity_tearout = bolt_tearout_unity(connection_properties)
  unity_net_section_tension = bolt_net_section_tension(connection_properties)
  unity_bearing = bolt_bearing(connection_properties)

  unity = max(unity_tearout, unity_net_section_tension, unity_bearing)

  return unity

### 5.3.2 Tearout

In [ ]:
def bolt_tearout_unity(connection_properties):
  # tearout capacity checked for both sections, per Section 5.3.2

  # get action
  V = connection_properties['V']

  # get connection properties
  t1 = connection_properties['t1']
  e1 = connection_properties['e1']
  fy1 = connection_properties['fy1']
  fu1 = connection_properties['fu1']
  t2 = connection_properties['t2']
  e2 = connection_properties['e2']
  fy2 = connection_properties['fy2']
  fu2 = connection_properties['fu2']

  # calculate unity for sheet 1, per eq 5.2.3(1)
  # calculate phi
  if fu1/fy1 >= 1.05:
    phi1 = 0.7
  elif fu1/fy1 < 1.05:
    phi1 = 0.6
  
  # calculate Vf per eq. 5.3.2(2)
  Vf1 = t1*e1*fu1

  # calculate unity for sheet 1
  unity1 = V/(phi1*Vf1)


  # calculate unity for sheet 2, per eq 5.2.3(1)
  # calculate phi
  if fu2/fy2 >= 1.05:
    phi2 = 0.7
  elif fu2/fy2 < 1.05:
    phi2 = 0.6
  
  # calculate Vf per eq. 5.3.2(2)
  Vf2 = t2*e2*fu2

  # calculate unity for sheet 2
  unity2 = V/(phi2*Vf2)


  # calculate worst case unity
  unity = max(unity1, unity2)

  return unity

### 5.3.3 Net section tension

In [ ]:
def bolt_net_section_tension(connection_properties):
  # net section tension capacity checked for both sections, per Section 5.3.3

  # get action
  V = connection_properties['V']
  
  # get connection properties
  df = connection_properties['connection size']
  t1 = connection_properties['t1']
  fu1 = connection_properties['fu1']
  An1 = connection_properties['An1']
  t2 = connection_properties['t2']
  fu2 = connection_properties['fu2']
  An2 = connection_properties['An2']

  # if fastener spacing perpendicular to shear force is not filled in, take the sheet widths perpendicular to shear force
  if (connection_properties['spacing'] == None) or (connection_properties['spacing'] == 0):
    s1 = connection_properties['w1']
    s2 = connection_properties['w2']
  else:
    s1 = connection_properties['spacing']
    s2 =  connection_properties['spacing']
  
  # get phi from Table 1.6.3 dataframe. .loc gets the series corresponding to 'Reference' = 5.3.3, .iloc[0] takes the first value of this series, which is the phi factor
  phi = table1_6_3.loc[table1_6_3['Reference'] == '5.3.3','Capacity Reduction Factor'].iloc[0]

  # calculate Nf per eq. 5.3.3(2) for each member, and take the lowest value
  Nf1 = (0.9 + (0.1*df/s1))*An1*fu1
  Nf2 = (0.9 + (0.1*df/s2))*An2*fu2
  Nf = min(Nf1, Nf2)

  unity = V/(phi*Nf)

  return unity

### 5.3.4 Bearing

This section inludes Table 5.3.4.2(A). This must be accessed by the engineering method, and the appropriate value of alpha provided as part of the `connection_properties` dict provided to the `bolt_bearing` function. 

#### Table 5.3.4.2(A)

In [ ]:
#@title Table 5.3.4.2(A) - Modification factor ($\alpha$) for type of bearing connection { vertical-output: true }

table5_3_4_2A = pd.DataFrame([
                                ['Single shear and outside sheets of double shear connection with washers under both bolt head and nut',1],
                                ['Single shear and outside sheets of double shear connection without washers under both bolt head and nut, or with only one washer',0.75],
                                ['Single shear and outside sheets of double shear connection using oversized or short-slotted holes parallel to the applied load without washers under both bolt head and nut, or with only one washer',0.7],
                                ['Single shear and outside sheets of double shear connection using oversized or short-slotted holes perpendicular to the applied load without washers under both bolt head and nut, or with only one washer',0.55],
                                ['Inside sheet of double shear connection with or without washers',1.33,],
                                ['Inside sheet of double shear connection using oversized or short-slotted holes parallel to the applied load with or without washers',1.10],
                                ['Inside sheet of double shear connection using oversized or short-slotted holes perpendicular to the applied load with or without washers',0.9]
                                ],
                                columns = ['Type of bearing','alpha']
                                )

table5_3_4_2A

,Type of bearing,alpha
0,Single shear and outside sheets of double shea...,1.00
1,Single shear and outside sheets of double shea...,0.75
2,Single shear and outside sheets of double shea...,0.70
3,Single shear and outside sheets of double shea...,0.55
4,Inside sheet of double shear connection with o...,1.33
5,Inside sheet of double shear connection using ...,1.10
6,Inside sheet of double shear connection using ...,0.90


In [ ]:
def bolt_bearing(connection_properties):
  # bolt bearing capacity checked for both sections, per Section 5.3.4
  
  # get action
  V = connection_properties['V']

  # get connection properties
  df = connection_properties['connection size']
  t1 = connection_properties['t1']
  fu1 = connection_properties['fu1']
  alpha1 = connection_properties['alpha1']
  t2 = connection_properties['t2']
  fu2 = connection_properties['fu2']
  alpha2 = connection_properties['alpha2']

  # define phi per Section 5.3.4.2
  phi = 0.6

  # calculate nominal bearing capacity for each sheet, per eq. 5.3.4.2
  # calculate bearing factor, C, per Tabke 5.3.4.2(B). Use a function as it is quite a large nested if statement and it is repeated for both sheets
  def bearing_factor(df, t):
    # initialise C as 0, so it will be 0 if thickness is out of bounds
    C = 0

    if t >= 0.42 and t < 4.76:
      if df/t < 10:
        C = 3
      elif df/t <= 22:
        C = 4-0.1*(df/t)
      else:
        C = 1.8
    
    return C

  C1 = bearing_factor(df, t1)
  C2 = bearing_factor(df, t2)

  Vb1 = alpha1 * C1 * df * t1 * fu1
  Vb2 = alpha2 * C2 * df * t2 * fu2
  Vb = min(Vb1, Vb2)

  unity = V/(phi*Vb)

  return unity

### Screwed connections

Not covered yet

# Appendix D

## Paragraph D1.1 Members in Compression - Global buckling stress

Used to calculate $f_{oc}$. The top level function simply determines whether D1.1 (sections without holes) or D1.2 (sections with holes) applies.

In [ ]:
def elastic_flexural_buckling_stress(section_properties,member_properties):
  if section_properties['hole sections']['holes present?'] == False:
    foc = elastic_flexural_buckling_stress_D_1_1_1(section_properties, member_properties)
  elif section_properties['hole sections']['holes present?'] == True:
    foc = elastic_flexural_buckling_stress_D_1_1_2(section_properties, member_properties)

  return foc

### D1.1.1 Compression members without holes

**D1.1.1.1 - Sections not subject to torsional or flexural-torsional buckling**

For sections not subject to distortional buckling, $f_{oc}$ is determined using eq D1.1.1(1):

>$f_{oc} = \frac{\pi^2E}{(\frac{l_e}{r})^2}
$

where:
>$l_e=$ effective length of member

>$r=$ radius of gyration of the full, unreduced cross section. 

>>$\qquad r=\sqrt{\frac{I_f}{A_f}}
$  

> although not explicitely required by the standard, the maximum of $\frac{l_{ey}}{r_y}$ and $\frac{l_{ex}}{r_x}$ is taken here

**D1.1.1.2 - Doubly or singly symmetric sections sunject to torsional or flexural-torsional buckling**

In this performance function all symmetricm sections are checked for torsional and flexural torsional buckling for simplicity and conservativeness.

For doubly or singly symmetric sections subject to torsional or flexural-torsional buckling $f_{oc}$ shall be taken as the smaller of $f_{oy}$, calculated using eq D1.1.1(4), and $f_{oxz}$, calcuated using eq. D1.1.1(2): 

>$f_{oxz} = \frac{1}{2\beta}[(f_{ox}+f_{oz})-\sqrt{(f_{ox}+f_{oz})^2-4\beta f_{ox}f_{oz}}\ ]
$

>$ f_{ox} = \frac{\pi^2E}{(\frac{l_{ex}}{r_x})^2}
$

>$ f_{oy} = \frac{\pi^2E}{(\frac{l_{ey}}{r_y})^2}
$

>$ f_{oz} = \frac{GJ}{A_gr_{ol}}(1 + \frac{\pi^2EI_w}{GJl_{ez}^2})
$

where:

>> $A_g =$ area of full cross section

>>$ r_{ol} =$ polar radius of gyration about the shear centre

>>$\quad\  = \sqrt{r_x^2+r_y^2+x_0^2+y_0^2}$

>>$ \beta = 1-(\frac{x_0}{r_{ol}})^2
$

**D1.1.1.3 - Point symmetric sections**

For point symmetruic sections subject to torsional or flexural-torsional buckling $f_{oc}$ shall be taken as the smaller of $f_{oc}$ as calulated using eq D1.1.1(1) and $f_{oz}$. Practically, this means:

>$ f_{oc} = min(f_{ox},f_{oy},f_{oz})
$

**D1.1.1.4 - Non-symmetric sections**

These are not currently covered by this performance function

In [ ]:
def elastic_flexural_buckling_stress_D_1_1_1(section_properties, member_properties):
  # D1.1.1.1 - Sections not subject to torsional or flexural-torsion buckling
  # NOTE: in this function, the x axis is taken to be the axis of symmetry for singly symmetric sections, and the major axis for other sections.
  # x,y axes within function may not match x,y input axes

  # get major/minor axis properties
  section_properties = axis_setter(section_properties)

  # get section properties, calculate radii of gyration
  Af = section_properties['Af']
  Ag = Af
  E = section_properties['E']
  G = section_properties['G']
  J = section_properties['J']
  Iw = section_properties['Iw']

  # per the D1.1.1.2, for singly symmetric sections, the x axis is taken to be the axis of symmetry
  # while not specifically outlined, the x axis is assumed to be the major axis in other parts of the standard, so the shear centre of the major axis is taken here for sections other than singly symmetric 
  if section_properties['symmetry axes'] == 'x' or section_properties['major_axis'] == 'x':
    Ixf = section_properties['Ix']
    Iyf = section_properties['Iy']
    x0 = section_properties['x0']
    y0 = section_properties['y0']
    # get effective lengths from member properties dictionary
    lex,ley,lez = member_properties['lex'],member_properties['ley'],member_properties['lez']

  elif section_properties['symmetry axes'] == 'y' or section_properties['major_axis'] == 'y':
    Iyf = section_properties['Ix']
    Ixf = section_properties['Iy']
    y0 = section_properties['x0']
    x0 = section_properties['y0']
    # get effective lengths from member properties dictionary
    ley,lex,lez = member_properties['lex'],member_properties['ley'],member_properties['lez']
  
  # calculate radii of gyration
  rx = (Ixf/Af)**0.5
  ry = (Iyf/Af)**0.5
  rol = (rx**2 + ry**2 + x0**2 + y0**2)**0.5

  # calculate fox, foy, foz using eqs D1.1.1(3),(4),(5)
  fox = (E*math.pi**2)/(lex/rx)**2
  foy = (E*math.pi**2)/(ley/ry)**2
  foz = (G*J/(Ag*rol**2))*(1+(E*Iw*math.pi**2)/(G*J*lez**2))

  # calculate beta using eq D1.1.1(7)
  beta = 1-(x0/rol)**2

  #calculate foxz using eq D1.1.1(2)
  foxz = (1/(2*beta)) * ( (fox + foz) - ((fox + foz)**2 -4*beta*fox*foz )**0.5 )

  # take foc as the minimum of fox, foy, foxz if section is singly or doubly symmetric.
  # take foc as the smaller of fox, foy, foz if section is point symmetric
  if section_properties['symmetry axes'] in {'x','y'}:
    foc = min(fox, foy, foxz)
  elif section_properties['symmetry axes'] in {'double','Z point symmetric'}:
    foc = min(fox, foy, foz)
  elif section_properties['symmetry axes'] in {'none'}:
    foc = 'error, performance function does not cover non-symmetric sections'

  return foc

###D1.1.2 Compression members with holes

$f_{oc}$ is calculated per equations D1.1.2(1)-(7). In general the same process is followed as for D1.1.1 - Compression members without holes.

However, where required weighted average section properties are taken from Table D1.1.2.1

In [ ]:
def weighted_average_section_properties(section_properties):
  # get relevant gross section properties from section_properties dictionary
  A_g = section_properties['Af']
  x0 = section_properties['x0']
  y0 = section_properties['y0']
  J = section_properties['J']
  Ix = section_properties['Ix']
  Iy = section_properties['Iy']
  
  # get relevant holed section properties from section_properties dictionary
  A_net = section_properties['hole sections']['A,net']
  Ix_net = section_properties['hole sections']['Ix,net']
  Iy_net = section_properties['hole sections']['Iy,net']
  J_net = section_properties['hole sections']['J,net']
  x0_net = section_properties['hole sections']['x0,net']
  y0_net = section_properties['hole sections']['y0,net']
  l_hole = section_properties['hole sections']['hole length']
  hole_spacing = section_properties['hole sections']['hole spacing']
  
  # calculate length of gross section (clear spacing between holes)
  l_g = hole_spacing-l_hole

  #calculate weighted average cross section properties per Table D1.1.2.1. For most properties this is done using a weighted average function, defined below
  def hole_weighted_averages(gross,net):
    avg = (gross*l_g+net*l_hole)/hole_spacing
    return avg

  # calculate weighted average section properties and add to section_properties dictionary
  A_avg = hole_weighted_averages(A_g,A_net)
  Ix_avg = hole_weighted_averages(Ix,Ix_net)
  Iy_avg = hole_weighted_averages(Iy,Iy_net)
  J_avg = hole_weighted_averages(J,J_net)
  x0_avg = hole_weighted_averages(x0,x0_net)
  y0_avg = hole_weighted_averages(y0,y0_net)
  rol_avg = (x0_avg**2 + y0_avg**2 + (Ix_avg+Iy_avg)/A_avg)**0.5

  return A_avg, Ix_avg, Iy_avg, J_avg, x0_avg, y0_avg, rol_avg

In [ ]:
def elastic_flexural_buckling_stress_D_1_1_2(section_properties, member_properties):
  #D1.1.1.1 - Sections not subject to torsional or flexural-torsion buckling

  #get major/minor axis properties
  section_properties = axis_setter(section_properties)

  # get section properties, calculate radii of gyration
  Ag = section_properties['Af']
  E = section_properties['E']
  G = section_properties['G']
  Iw_net = section_properties['hole sections']['Iw,net']

  # get weighted average section properties
  A_avg, Ix_avg, Iy_avg, J_avg, x0_avg, y0_avg, rol_avg = weighted_average_section_properties(section_properties)

  # get effective lengths from member properties dictionary
  lex,ley,lez = member_properties['lex'],member_properties['ley'],member_properties['lez']

  # per the D1.1.1.2, for singly symmetric sections, the x axis is taken to be the axis of symmetry
  # while not specifically outlined, the x axis is assumed to be the major axis in other parts of the standard, so the shear centre of the major axis is taken here for sections other than singly symmetric 
  # compared to the functioon for D1.1.1, in this function section properties come from a number of sources. So they are first set based on the input axes, and then x and y are switched below if necessary
  if section_properties['symmetry axes'] == 'y' or section_properties['major_axis'] == 'y':
    Ix_avg,Iy_avg = Iy_avg, Ix_avg
    x0_avg,y0_avg = y0_avg, x0_avg
    lex, ley = ley,lex

  # calculate fox, foy, foz using eqs D1.1.2(3),(4),(5)
  fox = (E*Ix_avg*math.pi**2)/(Ag*lex**2)
  foy = (E*Iy_avg*math.pi**2)/(Ag*ley**2)
  foz = (G*J_avg/(Ag*rol_avg**2))*(1+(E*Iw_net*math.pi**2)/(G*J_avg*lez**2))

  # calculate beta using eq D1.1.2(7). 
  beta = 1-(x0_avg/rol_avg)**2
  # print(f'beta = {beta}')

  #calculate foxz using eq D1.1.1(2)
  foxz = (1/(2*beta)) * ( (fox + foz) - ((fox + foz)**2 -4*beta*fox*foz )**0.5 )


  # print(f"fox = {fox}, foy = {foy}, foz = {foz}, foxz = {foxz}")
  # print(f"E = {E}, Ix_avg = {Ix_avg}, Ag = {Ag}, lex = {lex}")

  # take foc as the minimum of fox, foy, foxz if section is singly or doubly symmetric.
  # take foc as the smaller of fox, foy, foz if section is point symmetric
  if section_properties['symmetry axes'] in {'x','y'}:
    foc = min(fox, foy, foxz)
  elif section_properties['symmetry axes'] in {'double','Z point symmetric'}:
    foc = min(fox, foy, foz)
  elif section_properties['symmetry axes'] in {'none'}:
    foc = 'error, performance function does not cover non-symmetric sections'

  return foc

## Paragraph D1.2 Members in Compression - Distortional Buckling Stresses

Used to calculate $f_{od}$, for general channels and simple lipped channels (see Figure D2)

### D1.2.1.1 General channel members in compression

The elastic distortional buckling stress, $f_{od}$ is calculated using eqs D1.2.1(13)-(28).

This function also considers sections with patterned holes by using a modified thickness $t_r$ as per eq D1.2.2(2)

In [ ]:
def elastic_distortional_buckling_stress(section_properties):
  # calculate distortional buckling stress here using D1.2, D2.2
  return

## Paragraph D2.1 Global buckling moments

Used to calculate $M_o$

D2.1.1.2 *Single-, doubly- and point-symmetric sections* used to calculate $M_o$. 

D2.1.1.3 *Point symmetric Z-sections* not covered by performance function currently.

(a) **Singly-symmetric sections bent about the symmetry axis, doubly-symmetric sections bent about the x-axis, and Z-sections**
*For this section, the X-axis is taken as the major axis (axis with highest I value). This is not explicitly stated in the standard however is implied to be the case throughout.* 

>$ M_o = C_bA_gr_{ol}\sqrt{f_{oy}f_{oz}}
$

>> $C_b$ is conservatively taken as 1.0.

>> $r_{ol} = \sqrt{r_x^2 + r_y^2 + x_0^2 + y_0^2}
$
>>> $r_{x,y}$ are radii of gyration about respective axes

>>> $x_0,y_0$ are shear centre coordinates

>> $f_{oy}$, $f_{oz}$ calculated using equations D1.1.1(4) and D1.1.1(5) for non-holed sections, and equations D1.1.2(3) and D1.1.2(4) for holed sections.


In [ ]:
def elastic_lateral_torsional_buckling_moment(section_properties, member_properties):
  #NOTE: this function is only required to check major axis bending. Therefore, x,y axes in variable names within the function correspond to major, minor axes and NOT to input x,y axes
  
    # set major/minor axes
    section_properties = axis_setter(section_properties)
    symmetry_axes = section_properties['symmetry axes maj min']

    # assume moment axis is the major axis - as lateral torsional buckling unlikely to occur under minor axis bending
    moment_axis = 'maj'

    # get variables
    A_g = section_properties['Af']
    shear_centre_maj = section_properties['shear_centre_maj']
    shear_centre_min = section_properties['shear_centre_min']
    E = section_properties['E']
    G = section_properties['G']
    J = section_properties['J']
    Iw = section_properties['Iw']
    Imaj = section_properties['Imaj']
    Imin = section_properties['Imin']
    beta_y = section_properties['beta']
    r_maj = (Imaj/A_g)**0.5
    r_min = (Imin/A_g)**0.5

    if section_properties['major_axis'] == 'x':
      l_e_maj = member_properties['lex']
      l_e_min = member_properties['ley']
      l_e_torsion = member_properties['lez']
    elif section_properties['major_axis'] == 'y':
      l_e_maj = member_properties['ley']
      l_e_min = member_properties['lex']
      l_e_torsion = member_properties['lez']

    # define polar radius of gyration per eq D2.1.1(3)
    r_ol = (r_maj**2 + r_min**2 + shear_centre_maj**2 + shear_centre_min**2)**0.5

    # elastic buckling stresses per D1.1
    # non-holed sections
    if section_properties['hole sections']['holes present?'] == False:
      f_ox = ((math.pi**2)*E)  /  ((l_e_maj/r_maj)**2)
      f_oy = ((math.pi**2)*E)  /  ((l_e_min/r_min)**2)
      f_oz = ((G*J) / (A_g*r_ol**2)) * (1+(math.pi**2*E*Iw) / (G*J*l_e_torsion**2))

    # holed-sections
    elif section_properties['hole sections']['holes present?'] == True:

      # get holed section properties
      A_net = section_properties['hole sections']['A,net']
      Imaj_net = section_properties['hole sections']['Imaj,net']
      Imin_net = section_properties['hole sections']['Imin,net']
      Iw_net = section_properties['hole sections']['Iw,net']
      J_net = section_properties['hole sections']['J,net']
      shear_centre_maj_net = section_properties['hole sections']['shear_centre_maj,net']
      shear_centre_min_net = section_properties['hole sections']['shear_centre_min,net']
      l_hole = section_properties['hole sections']['hole length']
      hole_spacing = section_properties['hole sections']['hole spacing']
      l_g = hole_spacing-l_hole

      #calculate weighted average cross section properties per Table D1.1.2.1
      def hole_weighted_averages(gross,net):
        avg = (gross*l_g+net*l_hole)/hole_spacing
        return avg

      A_avg = hole_weighted_averages(A_g,A_net)
      Ix_avg = hole_weighted_averages(Imaj,Imaj_net)
      Imin_avg = hole_weighted_averages(Imin,Imin_net)
      J_avg = hole_weighted_averages(J,J_net)
      shear_centre_maj_avg = hole_weighted_averages(shear_centre_maj, shear_centre_maj_net)
      shear_centre_min_avg = hole_weighted_averages(shear_centre_min,shear_centre_min_net)
      r_ol_avg =  (shear_centre_maj_avg**2 + shear_centre_min_avg**2 + (Ix_avg+Imin_avg)/A_avg)**0.5

      #set effective length in minor axis
      if section_properties['major_axis'] == 'x':
        l_e_min = member_properties['ley']
      elif section_properties['major_axis'] == 'y':
        l_e_min = member_properties['lex']

      #set effective length for torsion
      l_e_torsion = member_properties['lez']

      f_oy = ((math.pi**2)*E*Imin_net)  /  (A_g*l_e_min**2)
      f_oz = ((G*J_avg) / (A_g*r_ol_avg**2)) * (1+(math.pi**2*E*Iw_net) / (G*J*l_e_torsion**2))

    # D2.1.1.2(a) if singly symmetric and bent about symmetry axis (that also being the major axis) or if doubly symmetric and bent about x (major) axis:
    if (symmetry_axes == 'maj' and moment_axis == 'maj') or (symmetry_axes == 'double' and moment_axis == 'maj'):
      C_b = 1.0
      M_o = C_b*A_g*r_ol*(f_oy*f_oz)**0.5

    # D2.1.1.2(b) if singly symmetric and bent about perpendicular axis
    if (symmetry_axes == 'min' and moment_axis == 'maj'):
      C_TF = 1                                                                                      #conservative, can be reduced if moment is greatest at ends of the member rather than midspan
      
      # calculate M_o for both the case where moment causes compression on the shear centre side of the section, and the case where moment causes tension on the shear centre side of the section. Take the lowest value. This will give a conservative result. 
      def M_o_D2_1_1_4(case):  
        if case == 1:
          # case 1, compression on shear centre side
          C_s = 1
          beta_y = abs(beta_y)
          M_o = C_s*A_g*f_ox*( (beta_y/2)+C_s*( (beta_y/2)**2 + (r_ol**2*f_oz/f_ox))**0.5 ) / C_TF
        elif case == 2:
          # case 2, tension on shear centre side
          C_s = -1
          beta_y = - abs(beta_y)
          M_o = C_s*A_g*f_ox*( (beta_y/2)+C_s*( (beta_y/2)**2 + (r_ol**2*f_oz/f_ox))**0.5 ) / C_TF
        return M_o
      
      # use M_o_D2_1_1_1_4 function to calculate M0 per equation D2.1.1(4), for both positive and negative moment
      M_o = min(M_o_D2_1_1_4(1),M_o_D2_1_1_4(2))

    return M_o